# Deep Learning Tutorial

#### Abram Hindle
#### <abram.hindle@ualberta.ca>
#### http://softwareprocess.ca/

Slides stolen gracefully from Ben Zittlau

Slide content under CC-BY-SA 4.0 and MIT License for source code or the same license as Python3 or Keras. Slide Source code is MIT License as well.




# Intro
### What is machine learning?

Building a function from data to classify, predict, group, or represent data.



# Intro
### Machine Learning

There are a few kinds of tasks or functions that could help us here.

* Classification: given some input, predict the class that it belongs
  to. Given a point is it in the red or in the blue?
* Regression: Given a point what will its value be? In the case of a
  function with a continuous or numerous discrete outputs it might be
  appropriate.
* Representation: Learn a smaller representation of the input
  data. E.g. we have 300 features lets describe them in a 128-bit hash.




# Intro
### Motivational Example

Imagine we have this data:

![2 crescent slices](images/slice.png "A function we want to learn
 f(x,y) -> z where z is red")

[See src/genslice.py to see how we made it.](src/genslice.py)



In [1]:
# purpose: make a slight difference of circles be the dataset to learn.
import numpy as np
gY, gX = np.meshgrid(np.arange(1000)/1000.0,np.arange(1000)/1000.0)

def intersect_circle(cx,cy,radius):
    equation = (gX - float(cx)) ** 2 + (gY - float(cy)) ** 2
    matches = equation < (radius**3) 
    return matches

# rad = 0.1643167672515498
rad = 0.3
x = intersect_circle(0.5,0.5,rad) ^ intersect_circle(0.51,0.51,rad)

def plotit(x):
    import matplotlib.pyplot as plt
    plt.imshow(x)
    plt.savefig('new-slice.png') # was slice.png
    plt.imshow(x)
    plt.savefig('new-slice.pdf') # was slice.pdf
    plt.show()

# plotit(x)

def mkcol(x):
    return x.reshape((x.shape[0]*x.shape[1],1))

# make the data set
big = np.concatenate((mkcol(gX),mkcol(gY),mkcol(1*x)),axis=1)
np.savetxt("new-big-slice.csv", big, delimiter=",")

# make a 50/50 data set
nots = big[big[0:,2]==0.0,]
np.random.shuffle(nots)
nots = nots[0:1000,]
trues = big[big[0:,2]==1.0,]
np.random.shuffle(trues)
trues = trues[0:1000,]
small = np.concatenate((trues,nots))
np.savetxt("new-small-slice.csv", small, delimiter=",")


# Intro
### Make your own function

``` python
def in_circle(x,y,cx,cy,radius):
    return (x - float(cx)) ** 2 + (y - float(cy)) ** 2 < radius**2

def mysolution(pt,outer=0.3):
    return in_circle(pt[0],pt[1],0.5,0.5,outer) and not in_circle(pt[0],pt[1],0.5,0.5,0.1)
```

```
>>> myclasses = np.apply_along_axis(mysolution,1,test[0])
>>> print "My classifier!"
My classifier!
>>> print "%s / %s " % (sum(myclasses == test[1]),len(test[1]))
181 / 200 
>>> print theautil.classifications(myclasses,test[1])
[('tp', 91), ('tn', 90), ('fp', 19), ('fn', 0)]
```



# Intro 
### An example classifier

1-NN: 1 Nearest Neighbor.

Given the data, we produce a function that
outputs the CLASS of the nearest neighbour to the input data.

Whoever is closer, is the class. 3-NN is 3-nearest neighbors whereby
we use voting of the 3 neighbors instead.



# Intro
### An example classifier: 1-NN

[src/slice-classifier.py](src/slice-classifier.py)

``` python
def euclid(pt1,pt2):
    return sum([ (pt1[i] - pt2[i])**2 for i in range(0,len(pt1)) ])

def oneNN(data,labels):
    def func(input):
        distance = None
        label = None
        for i in range(0,len(data)):
            d = euclid(input,data[i])
            if distance == None or d < distance:
                distance = d
                label = labels[i]
        return label
    return func
```




# Intro
### An example classifier: 1-NN

``` python
>>> learner = oneNN(train[0],train[1])
>>> 
>>> oneclasses = np.apply_along_axis(learner,1,test[0])
>>> print "1-NN classifier!"
1-NN classifier!
>>> print "%s / %s " % (sum(oneclasses == test[1]),len(test[1]))
198 / 200 
>>> print theautil.classifications(oneclasses,test[1])
[('tp', 91), ('tn', 107), ('fp', 2), ('fn', 0)]

```

1-NN has great performance in this example, but it uses Euclidean
distance and the dataset is really quite biased to the positive
classes.

Thus we showed a simple learner that classifies data.



# Intro

* That's really interesting performance and it worked but will it
  scale and continue to work?

* 1-NN doesn't work for all problems. And it is dependent on linear
  relationships.

* What if our problem is non-linear?




In [26]:
#
# The MIT License (MIT)
# 
# Copyright (c) 2016 Abram Hindle <hindle1@ualberta.ca>, Leif Johnson <leif@lmjohns3.com>
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# first off we load up some modules we want to use
import keras
import scipy
import math
import numpy as np
import numpy.random as rnd
import logging
import sys
import collections
import theautil

# setup logging
logging.basicConfig(stream = sys.stderr, level=logging.INFO)

mupdates = 1000
data = np.loadtxt("small-slice.csv", delimiter=",")
inputs  = data[0:,0:2].astype(np.float32)
outputs = data[0:,2:3].astype(np.int32)

theautil.joint_shuffle(inputs,outputs)

train_and_valid, test = theautil.split_validation(90, inputs, outputs)
train, valid = theautil.split_validation(90, train_and_valid[0], train_and_valid[1])
print("Train: %s Valid: %s"%(len(train),len(valid)))
print("Train X: %s Y^:%s"%(train[0].shape,train[1].shape))
print("Valid X: %s  Y^:%s"%(valid[0].shape,valid[1].shape))


Train: 2 Valid: 2
Train X: (1620, 2) Y^:(1620, 1)
Valid X: (180, 2)  Y^:(180, 1)


In [28]:

def linit(x):
    return x.reshape((len(x),))

mltrain = (train[0],linit(train[1]))
mlvalid = (valid[0],linit(valid[1]))
mltest  = (test[0] ,linit(test[1]))

# my solution
def in_circle(x,y,cx,cy,radius):
    return (x - float(cx)) ** 2 + (y - float(cy)) ** 2 < radius**2

def mysolution(pt,outer=0.3):
    return in_circle(pt[0],pt[1],0.5,0.5,outer) and \
           not in_circle(pt[0],pt[1],0.5,0.5,0.1)

# apply my classifier
myclasses = np.apply_along_axis(mysolution,1,mltest[0])
print("My classifier!")
print("%s / %s " % (sum(myclasses == mltest[1]),len(mltest[1])))
print(theautil.classifications(myclasses,mltest[1]))

My classifier!
174 / 200 
[('tp', 98), ('tn', 76), ('fp', 26), ('fn', 0)]


In [29]:
def euclid(pt1,pt2):
    return sum([ (pt1[i] - pt2[i])**2 for i in range(0,len(pt1)) ])

def oneNN(data,labels):
    def func(input):
        distance = None
        label = None
        for i in range(0,len(data)):
            d = euclid(input,data[i])
            if distance == None or d < distance:
                distance = d
                label = labels[i]
        return label
    return func

learner = oneNN(mltrain[0],mltrain[1])

oneclasses = np.apply_along_axis(learner,1,mltest[0])
print("1-NN classifier!")
print("%s / %s " % (sum(oneclasses == mltest[1]),len(mltest[1])))
print(theautil.classifications(oneclasses,mltest[1]))


1-NN classifier!
198 / 200 
[('tp', 98), ('tn', 100), ('fp', 2), ('fn', 0)]



# Intro

* Neural networks are popular
   * Creating AI for Go
   * Labeling Images with cats and dogs
   * Speech Recognition
   * Text summarization
   * [Guitar Transcription](https://peerj.com/preprints/1193.pdf)
   * Learn audio from video[1](https://archive.org/details/DeepLearningBitmaptoPCM/)[2](http://softwareprocess.es/blog/blog/2015/08/10/deep-learning-bitmaps-to-pcm/)

* Neural networks can not only classify, but they can create content,
  they can have complicated outputs.

* Neural networks are generative!


# Intro
### Machine Learning: Neural Networks

Neural networks or "Artificial Neural Networks" are a flexible class
of non-linear machine learners. They have been found to be quite
effective as of late.

Neural networks are composed of neurons. These neurons try to emulate
biological neurons in the most metaphorical of senses. Given a set of
inputs they produce an output.




## Neurons

Neurons have functions.

* Rectified Linear Units have been shown to train quite well and
  achieve good results. By they aren't easier to differentiate.
  f(x) = max(0,x)
* Sigmoid functions are slow and were the classical neural network
  neuron, but have fallen out of favour. They will work when nothing
  else will. f(x) = 1/(1 + e^-x)
* Softplus is a RELU that is slower to compute but differentiable.
  f(x) = ln(1 + e^x)




## Neurons

![Rectifier and Sigmoid and Softplus](images/Rectifier_and_softplus_functions.svg)






## Neurons

The inputs to a neural network? The outputs of connected nodes times
their weight + a bias.

neuron(inputs) = neuron_f( sum(weights * inputs) + bias  )

![Neuron example](images/neuron.png)




## Multi-layer perceptron

Single hidden layer neural network.

![Multi-layer perceptron](images/20160208141015.png)






## Deep Learning

There's nothing particularly crazy about deep learning other than it has more hidden layers.

These hidden layers allow it to compute state and address the intricacies of complex functions. But each hidden layer adds a lot of search space.




## Deep Learning

![Deep network, multiple layers](images/20160208141143.png)






## Search

How do we find the different weights?

Well we need to search a large space. A 2x3x2 network will have 2*3*2
weights + 5 biases (3 hidden, 2 output) resulting in 17
parameters. That's already a large search space.

Most search algorithms measure their error at a certain point
(difference between prediction and actual) and then choose a direction
in their search space to travel. They do this by sampling points
around themselves in order to compute a gradient or slope and then
follow the slope around.

Here's a 3D demo of different search algorithms.

[Different Search Parameters](http://www.robertsdionne.com/bouncingball/)





## Let's deep learn on our problem

![2 crescent slices](images/slice.png "A function we want to learn
 f(x,y) -> z where z is red")

Please open [slice-classifier](./src/slice-classifier.py) and a python
interpreter such as bpython. Search for Part 3 around line 100.




In [30]:

print('''
########################################################################
# Part 3. Let's start using neural networks!
########################################################################
''')

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train[1])
train_y = enc.transform(train[1])
valid_y = enc.transform(valid[1])
test_y = enc.transform(test[1])

print(train[0].shape)
print(train[1].shape)
print(train_y.shape)



########################################################################
# Part 3. Let's start using neural networks!
########################################################################

(1620, 2)
(1620, 1)
(1620, 2)


In [44]:
# rerunning this will produce different results
# try different combos here
net = Sequential()
net.add(Dense(16,input_shape=(2,),activation="sigmoid"))
net.add(Dense(32,activation="sigmoid"))
net.add(Dense(2,activation="softmax"))
# opt = SGD(lr=0.1)#
opt = Adam(lr=0.1)
net.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
history = net.fit(train[0], train_y, validation_data=(valid[0], valid_y),
	            epochs=100, batch_size=16)


Train on 1620 samples, validate on 180 samples
Epoch 1/100
1620/1620 [==============================] - 1s 447us/step - loss: 0.7105 - acc: 0.5130 - val_loss: 0.6953 - val_acc: 0.4667
Epoch 2/100
1620/1620 [==============================] - 0s 94us/step - loss: 0.7002 - acc: 0.5222 - val_loss: 0.6909 - val_acc: 0.5333
Epoch 3/100
1620/1620 [==============================] - 0s 105us/step - loss: 0.6965 - acc: 0.5019 - val_loss: 0.6995 - val_acc: 0.4667
Epoch 4/100
1620/1620 [==============================] - 0s 109us/step - loss: 0.7032 - acc: 0.4901 - val_loss: 0.6926 - val_acc: 0.5333
Epoch 5/100
1620/1620 [==============================] - 0s 103us/step - loss: 0.7047 - acc: 0.4852 - val_loss: 0.6993 - val_acc: 0.4667
Epoch 6/100
1620/1620 [==============================] - 0s 107us/step - loss: 0.6965 - acc: 0.4957 - val_loss: 0.6915 - val_acc: 0.5333
Epoch 7/100
1620/1620 [==============================] - 0s 105us/step - loss: 0.6272 - acc: 0.5889 - val_loss: 0.6282 - val_acc: 0.

In [45]:
print("Learner on the test set")
score = net.evaluate(test[0], test_y)
print("Scores: %s" % score)
predictit = net.predict(test[0])
print(predictit.shape)
print(predictit[0:10,])
classify = net.predict_classes(test[0])

print("%s / %s " % (np.sum(classify == mltest[1]),len(mltest[1])))
print(collections.Counter(classify))
print(theautil.classifications(classify,mltest[1]))


Learner on the test set
200/200 [==============================] - 0s 46us/step
Scores: [0.2429424160718918, 0.925]
(200, 2)
[[1.37688160e-01 8.62311900e-01]
 [9.99723494e-01 2.76449253e-04]
 [4.11696076e-01 5.88303924e-01]
 [1.21386774e-01 8.78613234e-01]
 [9.99723494e-01 2.76449253e-04]
 [1.08947322e-01 8.91052723e-01]
 [7.01258704e-02 9.29874063e-01]
 [7.64196962e-02 9.23580289e-01]
 [1.00000000e+00 4.84003066e-27]
 [7.61788711e-02 9.23821092e-01]]
185 / 200 
Counter({1: 111, 0: 89})
[('tp', 97), ('tn', 88), ('fp', 14), ('fn', 1)]


Let's try this on unseen data.

In [46]:

def real_function(pt):
    rad = 0.1643167672515498
    in1 = in_circle(pt[0],pt[1],0.5,0.5,rad)
    in2 = in_circle(pt[0],pt[1],0.51,0.51,rad)
    return in1 ^ in2

print("And now on more unseen data that isn't 50/50")

bigtest = np.random.uniform(size=(3000,2)).astype(np.float32)
biglab = np.apply_along_axis(real_function,1,bigtest).astype(np.int32)

classify = net.predict_classes(bigtest)
print("%s / %s " % (sum(classify == biglab),len(biglab)))
print(collections.Counter(classify))
print(theautil.classifications(classify,biglab))


And now on more unseen data that isn't 50/50
2698 / 3000 
Counter({0: 2663, 1: 337})
[('tp', 36), ('tn', 2662), ('fp', 301), ('fn', 1)]




## Now let's discuss posing problems for neural networks

* Scaling inputs: Scaling can sometimes help, so can
  standardization. This means constraining values or re-centering
  them. It depends on your problem and it is worth trying.

* E.g. min max scaling:

``` python
def min_max_scale(data):
    '''scales data by minimum and maximum values between 0 and 1'''
    dmin = np.min(data)
    return (data - dmin)/(np.max(data) - dmin)
```



## The problem

* [posing.py](src/posing.py) tries to show the problem of taking
  random input data and determine what distribution it comes from.
  That is what function can produce these random values.

* Let's open up [posing.py](src/posing.py) and get an interpreter
  going.



In [2]:
# Demonstration of how to pose the problem and how different formulations
# lead to different results!
#
# The MIT License (MIT)
# 
# Copyright (c) 2016 Abram Hindle <hindle1@ualberta.ca>, Leif Johnson <leif@lmjohns3.com>
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# first off we load up some modules we want to use
import keras
import scipy
import math
import numpy as np
import numpy.random as rnd
import logging
import sys
from numpy.random import power, normal, lognormal, uniform
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import theautil

# What are we going to do?
# - we're going to generate data derived from 4 different distributions
# - we're going to scale that data
# - we're going to create a RBM (1 hidden layer neural network)
# - we're going to train it to classify data as belonging to one of these distributions

# maximum number of iterations before we bail
mupdates = 1000

# setup logging
logging.basicConfig(stream = sys.stderr, level=logging.INFO)

# how we pose our problem to the deep belief network matters.

# lets make the task easier by scaling all values between 0 and 1
def min_max_scale(data):
    '''scales data by minimum and maximum values between 0 and 1'''
    dmin = np.min(data)
    return (data - dmin)/(np.max(data) - dmin)

# how many samples per each distribution
bsize    = 100 

# poor man's enum
LOGNORMAL=0
POWER=1
NORM=2
UNIFORM=3


Using TensorFlow backend.
/home/hindle1/.local/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)



## Experiment 1

* Given 1 single sample what distribution does it come from?




In [3]:
print('''
########################################################################
# Experiment 1: can we classify single samples?
#
#
#########################################################################
''')

def make_dataset1():
    '''Make a dataset of single samples with labels from which distribution they come from'''
    # now lets make some samples 
    lns      = min_max_scale(lognormal(size=bsize)) #log normal
    powers   = min_max_scale(power(0.1,size=bsize)) #power law
    norms    = min_max_scale(normal(size=bsize))    #normal
    uniforms = min_max_scale(uniform(size=bsize))    #uniform
    # add our data together
    data = np.concatenate((lns,powers,norms,uniforms))
    
    # concatenate our labels
    labels = np.concatenate((
        (np.repeat(LOGNORMAL,bsize)),
        (np.repeat(POWER,bsize)),
        (np.repeat(NORM,bsize)),
        (np.repeat(UNIFORM,bsize))))
    tsize = len(labels)
    
    # make sure dimensionality and types are right
    data = data.reshape((len(data),1))
    data = data.astype(np.float32)
    labels = labels.astype(np.int32)
    labels = labels.reshape((len(data),))
    
    return data, labels, tsize

# this will be the training data and validation data
data, labels, tsize = make_dataset1()


# this is the test data, this is kept separate to prove we can
# actually work on the data we claim we can.
#
# Without test data, you might just have great performance on the
# train set.
test_data, test_labels, _ = make_dataset1()



########################################################################
# Experiment 1: can we classify single samples?
#
#
#########################################################################



In [4]:
# utilities

# now lets shuffle
# If we're going to select a validation set we probably want to shuffle
def joint_shuffle(arr1,arr2):
    assert len(arr1) == len(arr2)
    indices = np.arange(len(arr1))
    np.random.shuffle(indices)
    arr1[0:len(arr1)] = arr1[indices]
    arr2[0:len(arr2)] = arr2[indices]

# our data and labels are shuffled together
joint_shuffle(data,labels)

def split_validation(percent, data, labels):
    ''' 
    split_validation splits a dataset of data and labels into
    2 partitions at the percent mark
    percent should be an int between 1 and 99
    '''
    s = int(percent * len(data) / 100)
    tdata = data[0:s]
    vdata = data[s:]
    tlabels = labels[0:s]
    vlabels = labels[s:]
    return ((tdata,tlabels),(vdata,vlabels))

# make a validation set from the train set
train1, valid1 = split_validation(90, data, labels)

print(train1[0].shape)
print(train1[1].shape)

enc1 = OneHotEncoder(handle_unknown='ignore')
enc1.fit(train1[1].reshape(len(train1[1]),1))
train1_y = enc1.transform(train1[1].reshape(len(train1[1]),1))
print(train1_y.shape)
valid1_y = enc1.transform(valid1[1].reshape(len(valid1[1]),1))
print(valid1_y.shape)
test1_y = enc1.transform(test_labels.reshape(len(test_labels),1))
print(test1_y.shape)

(360, 1)
(360,)
(360, 4)
(40, 4)
(400, 4)


In [5]:
# build our classifier

print("We're building a MLP of 1 input layer node, 4 hidden layer nodes, and an output layer of 4 nodes. The output layer has 4 nodes because we have 4 classes that the neural network will output.")
cnet = Sequential()
cnet.add(Dense(4,input_shape=(1,),activation="sigmoid"))
cnet.add(Dense(4,activation="softmax"))
copt = SGD(lr=0.1)
# opt = Adam(lr=0.1)
cnet.compile(loss="categorical_crossentropy", optimizer=copt, metrics=["accuracy"])
history = cnet.fit(train1[0], train1_y, validation_data=(valid1[0], valid1_y),
	            epochs=100, batch_size=16)

#score = cnet.evaluate(test_data, test_labels)
#print("Scores: %s" % score)
classify = cnet.predict_classes(test_data)
print(theautil.classifications(classify,test_labels))
score = cnet.evaluate(test_data, test1_y)
print("Scores: %s" % score)


We're building a MLP of 1 input layer node, 4 hidden layer nodes, and an output layer of 4 nodes. The output layer has 4 nodes because we have 4 classes that the neural network will output.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 360 samples, validate on 40 samples
Epoch 1/100
360/360 [==============================] - 0s 758us/step - loss: 1.4490 - acc: 0.2472 - val_loss: 1.4021 - val_acc: 0.1250
Epoch 2/100
360/360 [==============================] - 0s 175us/step - loss: 1.4101 - acc: 0.1250 - val_loss: 1.4029 - val_acc: 0.0250
Epoch 3/100
360/360 [==============================] - 0s 224us/step - loss: 1.4032 - acc: 0.1194 - val_loss: 1.4077 - val_acc: 0.0750
Epoch 4/100
360/360 [==============================] - 0s 219us/step - loss: 1.3988 - acc: 0.1389 - val_loss: 1.4083 - val_acc: 0.2000
Epoch 5/100
360/360 [==============================] - 0s 165us/step - loss: 1.3950 - acc: 0.2278 - val_loss: 1.4004 - val_acc: 0.1750
Epoch 6/100
360/360 [==============================] - 0s 166us/step - loss: 1.3926 - acc: 0.2500 - val_loss: 1.3918 - val_acc: 0.0750
Epoch 7/100
360/360 [==============================] - 0s 240us/step - loss: 1.3884 - acc: 0.1500 - val_loss: 1.4012 - val_acc: 0.2750
Epoch 8/10

Epoch 61/100
360/360 [==============================] - 0s 153us/step - loss: 1.0970 - acc: 0.4528 - val_loss: 1.0749 - val_acc: 0.5250
Epoch 62/100
360/360 [==============================] - 0s 125us/step - loss: 1.0942 - acc: 0.5028 - val_loss: 1.0695 - val_acc: 0.4500
Epoch 63/100
360/360 [==============================] - 0s 132us/step - loss: 1.0935 - acc: 0.4917 - val_loss: 1.0632 - val_acc: 0.4500
Epoch 64/100
360/360 [==============================] - 0s 120us/step - loss: 1.0897 - acc: 0.5000 - val_loss: 1.0633 - val_acc: 0.5000
Epoch 65/100
360/360 [==============================] - 0s 141us/step - loss: 1.0888 - acc: 0.4278 - val_loss: 1.0575 - val_acc: 0.5000
Epoch 66/100
360/360 [==============================] - 0s 135us/step - loss: 1.0860 - acc: 0.4500 - val_loss: 1.0572 - val_acc: 0.5250
Epoch 67/100
360/360 [==============================] - 0s 139us/step - loss: 1.0847 - acc: 0.4972 - val_loss: 1.0630 - val_acc: 0.4750
Epoch 68/100
360/360 [==========================


## Experiment 2

* Given 40 samples what distribution does it come from?



In [6]:
print('''
########################################################################
# Experiment 2: can we classify a sample of data?
#
#
#########################################################################
''')
print("In this example we're going to input 40 values from a single distribution, and we'll see if we can classify the distribution.")

width=40

def make_widedataset(width=width):
    # we're going to make rows of 40 features unsorted
    wlns      = min_max_scale(lognormal(size=(bsize,width))) #log normal
    wpowers   = min_max_scale(power(0.1,size=(bsize,width))) #power law
    wnorms    = min_max_scale(normal(size=(bsize,width)))    #normal
    wuniforms = min_max_scale(uniform(size=(bsize,width)))    #uniform
    
    wdata = np.concatenate((wlns,wpowers,wnorms,wuniforms))
    
    # concatenate our labels
    wlabels = np.concatenate((
        (np.repeat(LOGNORMAL,bsize)),
        (np.repeat(POWER,bsize)),
        (np.repeat(NORM,bsize)),
        (np.repeat(UNIFORM,bsize))))
    
    joint_shuffle(wdata,wlabels)
    wdata = wdata.astype(np.float32)
    wlabels = wlabels.astype(np.int32)
    wlabels = wlabels.reshape((len(data),))
    return wdata, wlabels

# make our train sets
wdata, wlabels = make_widedataset()
# make our test sets
test_wdata, test_wlabels = make_widedataset()

# split out our validation set
wtrain, wvalid = split_validation(90, wdata, wlabels)
print("At this point we have a weird decision to make, how many neurons in the hidden layer?")

encwc = OneHotEncoder(handle_unknown='ignore')
encwc.fit(wtrain[1].reshape(len(wtrain[1]),1))
wtrain_y = encwc.transform(wtrain[1].reshape(len(wtrain[1]),1))
wvalid_y = encwc.transform(wvalid[1].reshape(len(wvalid[1]),1))
wtest_y  = encwc.transform(test_wlabels.reshape(len(test_wlabels),1))

# wcnet = theanets.Classifier([width,width/4,4]) #267
wcnet = Sequential()
wcnet.add(Dense(width,input_shape=(width,),activation="sigmoid"))
wcnet.add(Dense(int(width/4),activation="sigmoid"))
wcnet.add(Dense(4,activation="softmax"))
wcnet.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.1), metrics=["accuracy"])
history = wcnet.fit(wtrain[0], wtrain_y, validation_data=(wvalid[0], wvalid_y),
	            epochs=100, batch_size=16)
score = wcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)




########################################################################
# Experiment 2: can we classify a sample of data?
#
#
#########################################################################

In this example we're going to input 40 values from a single distribution, and we'll see if we can classify the distribution.
At this point we have a weird decision to make, how many neurons in the hidden layer?
Train on 360 samples, validate on 40 samples
Epoch 1/100
360/360 [==============================] - 0s 898us/step - loss: 1.4187 - acc: 0.1917 - val_loss: 1.3722 - val_acc: 0.2750
Epoch 2/100
360/360 [==============================] - 0s 153us/step - loss: 1.3767 - acc: 0.3250 - val_loss: 1.3863 - val_acc: 0.4000
Epoch 3/100
360/360 [==============================] - 0s 252us/step - loss: 1.3597 - acc: 0.3583 - val_loss: 1.3547 - val_acc: 0.3500
Epoch 4/100
360/360 [==============================] - 0s 204us/step - loss: 1.3370 - acc: 0.3528 - val_loss: 1.3501 - val_acc: 0.1500


Epoch 58/100
360/360 [==============================] - 0s 180us/step - loss: 0.6087 - acc: 0.6833 - val_loss: 0.6055 - val_acc: 0.6250
Epoch 59/100
360/360 [==============================] - 0s 149us/step - loss: 0.6039 - acc: 0.6833 - val_loss: 0.6006 - val_acc: 0.7000
Epoch 60/100
360/360 [==============================] - 0s 144us/step - loss: 0.6069 - acc: 0.6472 - val_loss: 0.5891 - val_acc: 0.7250
Epoch 61/100
360/360 [==============================] - 0s 129us/step - loss: 0.6028 - acc: 0.6806 - val_loss: 0.5931 - val_acc: 0.7500
Epoch 62/100
360/360 [==============================] - 0s 130us/step - loss: 0.6007 - acc: 0.6972 - val_loss: 0.5912 - val_acc: 0.7000
Epoch 63/100
360/360 [==============================] - 0s 134us/step - loss: 0.5950 - acc: 0.7167 - val_loss: 0.5947 - val_acc: 0.6750
Epoch 64/100
360/360 [==============================] - 0s 149us/step - loss: 0.5930 - acc: 0.6833 - val_loss: 0.5896 - val_acc: 0.6250
Epoch 65/100
360/360 [==========================

In [7]:

classify = wcnet.predict_classes(test_wdata)
print(theautil.classifications(classify,test_wlabels))
score = wcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)

# # You could try some of these alternative setups
# 
# [width,4]) #248
# [width,width/2,4]) #271
# [width,width,4]) #289
# [width,width*2,4]) #292
# [width,width/2,width/4,4]) #270
# [width,width/2,width/4,width/8,width/16,4]) #232
# [width,width*8,4]) #304

print("Ok that was neat, it definitely worked better, it had more data though.")

print("But what if we help it out, and we sort the values so that the first and last bins are always the min and max values?")


[('tp', 81), ('tn', 96), ('fp', 4), ('fn', 19)]
400/400 [==============================] - 0s 68us/step
Scores: [0.5411265873908997, 0.6925]
Ok that was neat, it definitely worked better, it had more data though.
But what if we help it out, and we sort the values so that the first and last bins are always the min and max values?




## Experiment 3

* Given 40 sorted samples what distribution does it come from?


In [8]:
print('''
########################################################################
# Experiment 3: can we classify a SORTED sample of data?
#
#
#########################################################################
''')


print("Sorting the data")
wdata.sort(axis=1)
test_wdata.sort(axis=1)


swcnet = Sequential()
swcnet.add(Dense(width,input_shape=(width,),activation="sigmoid"))
swcnet.add(Dense(int(width/4),activation="sigmoid"))
swcnet.add(Dense(4,activation="softmax"))
swcnet.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.1), metrics=["accuracy"])
history = swcnet.fit(wtrain[0], wtrain_y, validation_data=(wvalid[0], wvalid_y),
	            epochs=100, batch_size=16)
score = swcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)




########################################################################
# Experiment 3: can we classify a SORTED sample of data?
#
#
#########################################################################

Sorting the data
Train on 360 samples, validate on 40 samples
Epoch 1/100
360/360 [==============================] - 0s 1ms/step - loss: 1.3864 - acc: 0.2944 - val_loss: 1.4028 - val_acc: 0.1500
Epoch 2/100
360/360 [==============================] - 0s 169us/step - loss: 1.3713 - acc: 0.2944 - val_loss: 1.3879 - val_acc: 0.2750
Epoch 3/100
360/360 [==============================] - 0s 156us/step - loss: 1.3613 - acc: 0.3472 - val_loss: 1.3476 - val_acc: 0.5250
Epoch 4/100
360/360 [==============================] - 0s 206us/step - loss: 1.3429 - acc: 0.4472 - val_loss: 1.3309 - val_acc: 0.4750
Epoch 5/100
360/360 [==============================] - 0s 243us/step - loss: 1.3255 - acc: 0.4500 - val_loss: 1.3121 - val_acc: 0.4750
Epoch 6/100
360/360 [==============================] - 

360/360 [==============================] - 0s 130us/step - loss: 0.1584 - acc: 0.9778 - val_loss: 0.1342 - val_acc: 0.9750
Epoch 60/100
360/360 [==============================] - 0s 207us/step - loss: 0.1533 - acc: 0.9806 - val_loss: 0.1312 - val_acc: 0.9750
Epoch 61/100
360/360 [==============================] - 0s 141us/step - loss: 0.1483 - acc: 0.9806 - val_loss: 0.1267 - val_acc: 0.9750
Epoch 62/100
360/360 [==============================] - 0s 132us/step - loss: 0.1434 - acc: 0.9806 - val_loss: 0.1242 - val_acc: 0.9750
Epoch 63/100
360/360 [==============================] - 0s 151us/step - loss: 0.1402 - acc: 0.9806 - val_loss: 0.1197 - val_acc: 0.9750
Epoch 64/100
360/360 [==============================] - 0s 129us/step - loss: 0.1351 - acc: 0.9833 - val_loss: 0.1180 - val_acc: 0.9750
Epoch 65/100
360/360 [==============================] - 0s 132us/step - loss: 0.1316 - acc: 0.9806 - val_loss: 0.1138 - val_acc: 0.9750
Epoch 66/100
360/360 [==============================] - 0s 14

In [9]:

classify = swcnet.predict_classes(test_wdata)
print(theautil.classifications(classify,test_wlabels))
score = swcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)


[('tp', 97), ('tn', 94), ('fp', 6), ('fn', 3)]
400/400 [==============================] - 0s 42us/step
Scores: [0.07569078877568244, 0.9775]


That was an improvement. What if we do binning instead?

## Experiment 4

* Given 40 histogrammed samples what distribution does it come from?



In [12]:
print('''
########################################################################
# Experiment 4: can we classify a discretized histogram of sample data?
#
#
#########################################################################
'''
)
# let's try actual binning
import collections

def bin(row):
    return np.histogram(row,bins=len(row),range=(0.0,1.0))[0]/float(len(row))

print("Apply the histogram to all the data rows")
bdata = np.apply_along_axis(bin,1,wdata).astype(np.float32)
blabels = wlabels

# ensure we have our test data
test_bdata = np.apply_along_axis(bin,1,test_wdata).astype(np.float32)
test_blabels = test_wlabels

# helper data 
enum_funcs = [
    (LOGNORMAL,"log normal",lambda size: lognormal(size=size)),
    (POWER,"power",lambda size: power(0.1,size=size)),
    (NORM,"normal",lambda size: normal(size=size)),
    (UNIFORM,"uniforms",lambda size: uniform(size=size)),
]

# uses enum_funcs to evaluate PER CLASS how well our classify operates
def classify_test(bnet,ntests=1000):
    for tup in enum_funcs:
        enum, name, func = tup
        lns = min_max_scale(func(size=(ntests,width))) #log normal
        blns = np.apply_along_axis(bin,1,lns).astype(np.float32)
        blns_labels = np.repeat(enum,ntests)
        blns_labels.astype(np.int32)
        classification = bnet.predict_classes(blns)
        classified = theautil.classifications(classification,blns_labels)
        print("Name:%s Tests:[%s] Count:%s -- Res:%s" % (name,ntests, collections.Counter(classification),classified ))

# train & valid
btrain, bvalid = split_validation(90, bdata, blabels)

encb = OneHotEncoder(handle_unknown='ignore')
encb.fit(btrain[1].reshape(len(btrain[1]),1))
btrain_y = encb.transform(btrain[1].reshape(len(btrain[1]),1))
bvalid_y = encb.transform(bvalid[1].reshape(len(bvalid[1]),1))
btest_y  = encb.transform(test_blabels.reshape(len(test_blabels),1))



# similar network structure
# bnet = theanets.Classifier([width,width/2,4])

bnet = Sequential()
bnet.add(Dense(width,input_shape=(width,),activation="sigmoid"))
bnet.add(Dense(int(width/4),activation="sigmoid"))
bnet.add(Dense(4,activation="softmax"))
bnet.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.1), metrics=["accuracy"])
history = bnet.fit(btrain[0], btrain_y, validation_data=(bvalid[0], bvalid_y),
	            epochs=100, batch_size=16)
score = bnet.evaluate(test_bdata, btest_y)
print("Scores: %s" % score)




########################################################################
# Experiment 4: can we classify a discretized histogram of sample data?
#
#
#########################################################################

Apply the histogram to all the data rows
Train on 360 samples, validate on 40 samples
Epoch 1/100
360/360 [==============================] - 0s 1ms/step - loss: 1.4001 - acc: 0.2417 - val_loss: 1.3716 - val_acc: 0.3250
Epoch 2/100
360/360 [==============================] - 0s 247us/step - loss: 1.3895 - acc: 0.2639 - val_loss: 1.3865 - val_acc: 0.2750
Epoch 3/100
360/360 [==============================] - 0s 229us/step - loss: 1.3923 - acc: 0.2278 - val_loss: 1.3920 - val_acc: 0.1500
Epoch 4/100
360/360 [==============================] - 0s 191us/step - loss: 1.3896 - acc: 0.2417 - val_loss: 1.3902 - val_acc: 0.2750
Epoch 5/100
360/360 [==============================] - 0s 204us/step - loss: 1.3914 - acc: 0.2611 - val_loss: 1.3965 - val_acc: 0.1500
Epoch 6/100
360/

360/360 [==============================] - 0s 164us/step - loss: 0.8743 - acc: 0.6222 - val_loss: 0.8392 - val_acc: 0.8500
Epoch 60/100
360/360 [==============================] - 0s 156us/step - loss: 0.8576 - acc: 0.5722 - val_loss: 0.8336 - val_acc: 0.8250
Epoch 61/100
360/360 [==============================] - 0s 142us/step - loss: 0.8340 - acc: 0.6194 - val_loss: 0.8375 - val_acc: 0.4000
Epoch 62/100
360/360 [==============================] - 0s 143us/step - loss: 0.8192 - acc: 0.5833 - val_loss: 0.8164 - val_acc: 0.5750
Epoch 63/100
360/360 [==============================] - 0s 147us/step - loss: 0.8049 - acc: 0.6194 - val_loss: 0.7857 - val_acc: 0.8750
Epoch 64/100
360/360 [==============================] - 0s 141us/step - loss: 0.7894 - acc: 0.6833 - val_loss: 0.7837 - val_acc: 0.5750
Epoch 65/100
360/360 [==============================] - 0s 148us/step - loss: 0.7791 - acc: 0.6583 - val_loss: 0.7730 - val_acc: 0.5000
Epoch 66/100
360/360 [==============================] - 0s 20

In [13]:

classify = bnet.predict_classes(test_bdata)
print(theautil.classifications(classify,test_blabels))
score = bnet.evaluate(test_bdata, btest_y)
print("Scores: %s" % score)

classify_test(bnet)

[('tp', 90), ('tn', 95), ('fp', 5), ('fn', 10)]
400/400 [==============================] - 0s 90us/step
Scores: [0.5862575101852417, 0.92]
Name:log normal Tests:[1000] Count:Counter({0: 923, 1: 77}) -- Res:[('tp', 0), ('tn', 923), ('fp', 77), ('fn', 0)]
Name:power Tests:[1000] Count:Counter({1: 857, 0: 143}) -- Res:[('tp', 857), ('tn', 0), ('fp', 0), ('fn', 143)]
Name:normal Tests:[1000] Count:Counter({2: 1000}) -- Res:[('tp', 0), ('tn', 0), ('fp', 0), ('fn', 0)]
Name:uniforms Tests:[1000] Count:Counter({3: 843, 2: 157}) -- Res:[('tp', 0), ('tn', 0), ('fp', 0), ('fn', 0)]


## Representation: Inputs

* For discrete values consider discrete inputs neurons. E.g. if you have 3 letters are your input you should have 3 * 26 input neurons. 
* Each neuron is "one-hot" -- 1 neuron is set to 1 to indicate that 1 discerete value. 
* An input of AAA would be: 
  * 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
* ZZZ would be 
  * 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1





## Representation: Inputs

* For groups of elements consider representing them as their counts.
* E.g. 3 cats, 4 dogs, 1 car as: 3 4 1 on 3 input neurons.
* Neural networks work well with distributions as inputs and distributions as outputs



## Representation: Words

* Words can be represented as word counts where by your vector is the count of each word per document -- you might have a large vocabulary so watch out!
* n-grams are popular too with one-hot encoding
* Embeddings (a dense vector representation) are popular too. Autoencoded words!



## Representaiton: Images

* Each neuron can represent a pixel represented from 0 to 1
* You can have images as output too!





## Representation: Outputs

* Do not ask the neural network to distingush discrete values on 1 neuron. Don't expect 1 neuron to output 0.25 for A and 0.9 for B and 1.0 for C. Use 3 neurons!
* Distribution outputs are good
* Interpretting the output is fine for regression problems



# Tuning

* The parameters you chose were probably not correct!
* You could grid search, that is try all combinations. But that takes a lot of time.
* We want to get engage in hyper-parameter tuning.
  * We want to find good parameters for our network that perform well.
* Grid Search
  * Step 1: choose the parameter space
  * Step 2: choose a method of selecting parameters
  * Step 3: get next combination of parameters
  * Step 4: evaluate the parameters
  * Step 5: If current performance is better than prior performances keep this set of parameters
  * Step 6: goto step 3 until all parameter combinations are exhausted.
  * Step 7: report results
* Random Search
  * Step 1: choose the parameter space
  * Step 2: choose a method of selecting parameters
  * Step 3: randomly choose parameters
  * Step 4: Evaluate the parameters
  * Step 5: If current performance is better than prior performances keep this set of parameters
  * Step 6: goto step 3 until satisfied (N iterations or M seconds)
  * Step 7: report results
  

In [14]:
# let's tune
print('''
########################################################################
# Experiment 5: Can we tune the binned data?
#
#
#########################################################################
'''
)


import Search

# 1 repetition
state = {"reps":1}
params = {"batch_size":[1,4,8,16,32,64],
          "lr":[1.0,0.1,0.01,0.001,0.0001],
          "activation":["sigmoid","tanh","relu"],
          "optimizer":["SGD","Adam"],
          "epochs":[25],
          "arch":[
              [width],
              [width,width],
              [width,int(width/4)],
              [2*width,width],
              [int(width/4),int(width/8)],
              [int(width/4)]]
          }
def get_optimizer(x):
    if x == "Adam":
        return Adam
    return SGD
    
def f(state,params):
    bnet = Sequential()
    arch = params["arch"]
    bnet.add(Dense(arch[0],input_shape=(width,),activation=params["activation"]))
    for layer in arch[1:]:
        bnet.add(Dense(int(layer),activation=params["activation"]))
    bnet.add(Dense(4,activation="softmax"))
    optimizer = get_optimizer(params["optimizer"])
    bnet.compile(loss="categorical_crossentropy",
                 optimizer=optimizer(lr=params["lr"]), metrics=["accuracy"])
    history = bnet.fit(btrain[0], btrain_y,
                       validation_data=(bvalid[0], bvalid_y),
	               epochs=params["epochs"], batch_size=params["batch_size"])
    classify = bnet.predict_classes(test_bdata)
    print(theautil.classifications(classify,test_blabels))
    score = bnet.evaluate(test_bdata, btest_y)
    print("Scores: %s" % score)
    return score[1]



########################################################################
# Experiment 5: Can we tune the binned data?
#
#
#########################################################################



In [15]:
# set the heuristic function to f
state["f"] = f
# random search for 60 seconds
random_results = Search.random_search(state,params,Search.heuristic_function,time=60)
# get the random results
random_results   = sorted(random_results, key=lambda x: x['Score'])
print(random_results[-1])


Train on 360 samples, validate on 40 samples
Epoch 1/25
360/360 [==============================] - 0s 1ms/step - loss: 1.1947 - acc: 0.4667 - val_loss: 0.9514 - val_acc: 0.6500
Epoch 2/25
360/360 [==============================] - 0s 55us/step - loss: 0.8857 - acc: 0.5639 - val_loss: 0.7574 - val_acc: 0.6250
Epoch 3/25
360/360 [==============================] - 0s 80us/step - loss: 0.7388 - acc: 0.6472 - val_loss: 0.7192 - val_acc: 0.6750
Epoch 4/25
360/360 [==============================] - 0s 58us/step - loss: 0.6328 - acc: 0.7778 - val_loss: 0.6396 - val_acc: 0.9000
Epoch 5/25
360/360 [==============================] - 0s 60us/step - loss: 0.6173 - acc: 0.7417 - val_loss: 0.5878 - val_acc: 0.7750
Epoch 6/25
360/360 [==============================] - 0s 48us/step - loss: 0.5299 - acc: 0.8639 - val_loss: 0.5772 - val_acc: 0.8500
Epoch 7/25
360/360 [==============================] - 0s 59us/step - loss: 0.5360 - acc: 0.7556 - val_loss: 0.5253 - val_acc: 0.8500
Epoch 8/25
360/360 [=====

360/360 [==============================] - 0s 73us/step - loss: 1.3530 - acc: 0.2500 - val_loss: 1.3388 - val_acc: 0.3250
Epoch 10/25
360/360 [==============================] - 0s 82us/step - loss: 1.3465 - acc: 0.2806 - val_loss: 1.3321 - val_acc: 0.3750
Epoch 11/25
360/360 [==============================] - 0s 85us/step - loss: 1.3399 - acc: 0.3278 - val_loss: 1.3241 - val_acc: 0.3750
Epoch 12/25
360/360 [==============================] - 0s 60us/step - loss: 1.3328 - acc: 0.3694 - val_loss: 1.3154 - val_acc: 0.4500
Epoch 13/25
360/360 [==============================] - 0s 68us/step - loss: 1.3249 - acc: 0.4056 - val_loss: 1.3068 - val_acc: 0.4750
Epoch 14/25
360/360 [==============================] - 0s 60us/step - loss: 1.3169 - acc: 0.4556 - val_loss: 1.2966 - val_acc: 0.5250
Epoch 15/25
360/360 [==============================] - 0s 52us/step - loss: 1.3075 - acc: 0.5139 - val_loss: 1.2865 - val_acc: 0.5500
Epoch 16/25
360/360 [==============================] - 0s 65us/step - loss

Epoch 18/25
360/360 [==============================] - 1s 3ms/step - loss: 0.3326 - acc: 0.9917 - val_loss: 0.3565 - val_acc: 1.0000
Epoch 19/25
360/360 [==============================] - 1s 3ms/step - loss: 0.2971 - acc: 0.9944 - val_loss: 0.3248 - val_acc: 0.9750
Epoch 20/25
360/360 [==============================] - 1s 2ms/step - loss: 0.2650 - acc: 0.9944 - val_loss: 0.2971 - val_acc: 0.9750
Epoch 21/25
360/360 [==============================] - 1s 3ms/step - loss: 0.2369 - acc: 0.9889 - val_loss: 0.2711 - val_acc: 1.0000
Epoch 22/25
360/360 [==============================] - 1s 3ms/step - loss: 0.2121 - acc: 0.9944 - val_loss: 0.2479 - val_acc: 1.0000
Epoch 23/25
360/360 [==============================] - 1s 2ms/step - loss: 0.1899 - acc: 0.9917 - val_loss: 0.2283 - val_acc: 1.0000
Epoch 24/25
360/360 [==============================] - 1s 3ms/step - loss: 0.1703 - acc: 0.9944 - val_loss: 0.2101 - val_acc: 1.0000
Epoch 25/25
360/360 [==============================] - 1s 3ms/step - 

400/400 [==============================] - 0s 48us/step
Scores: [0.615139970779419, 0.74]
{'batch_size': 1, 'lr': 0.0001, 'activation': 'relu', 'optimizer': 'Adam', 'epochs': 25, 'arch': [40, 10], 'Score': 0.9975}


In [16]:
print(random_results[-10:]) # print last 10 results

[{'batch_size': 32, 'lr': 0.001, 'activation': 'sigmoid', 'optimizer': 'Adam', 'epochs': 25, 'arch': [10], 'Score': 0.5275}, {'batch_size': 64, 'lr': 0.001, 'activation': 'relu', 'optimizer': 'Adam', 'epochs': 25, 'arch': [10, 5], 'Score': 0.6725}, {'batch_size': 8, 'lr': 0.1, 'activation': 'sigmoid', 'optimizer': 'SGD', 'epochs': 25, 'arch': [40], 'Score': 0.74}, {'batch_size': 1, 'lr': 0.01, 'activation': 'sigmoid', 'optimizer': 'SGD', 'epochs': 25, 'arch': [80, 40], 'Score': 0.765}, {'batch_size': 64, 'lr': 1.0, 'activation': 'tanh', 'optimizer': 'SGD', 'epochs': 25, 'arch': [10], 'Score': 0.995}, {'batch_size': 64, 'lr': 0.1, 'activation': 'relu', 'optimizer': 'Adam', 'epochs': 25, 'arch': [40], 'Score': 0.9975}, {'batch_size': 1, 'lr': 0.0001, 'activation': 'relu', 'optimizer': 'Adam', 'epochs': 25, 'arch': [40, 10], 'Score': 0.9975}]



## References

* [Theanets Documentation](https://theanets.readthedocs.org/en/stable/)
* [A Practical Guide to TrainingRestricted Boltzmann Machines](https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf)
* [MLP](http://deeplearning.net/tutorial/mlp.html#mlp)
* [Deep Learning Tutorials](http://www.iro.umontreal.ca/~pift6266/H10/notes/deepintro.html)
* [Deep Learning Tutorials](http://deeplearning.net/tutorial/)
* [Coursera: Hinton's Neural Networks for Machine Learning](https://www.coursera.org/course/neuralnets)
* [The Next Generation of Neural Networks](https://www.youtube.com/watch?v=AyzOUbkUf3M)
* [Geoffrey Hinton: "Introduction to Deep Learning & Deep Belief Nets"](https://www.youtube.com/watch?v=GJdWESd543Y)
* Bengio's Deep Learning
  [(1)](https://www.youtube.com/watch?v=JuimBuvEWBg)[(2)](https://www.youtube.com/watch?v=Fl-W7_z3w3o)
* [Nvidia's Deep Learning tutorials](https://developer.nvidia.com/deep-learning-courses
)
* [Udacity Deep Learning MOOC](https://www.udacity.com/course/deep-learning--ud730)
